In [1]:
# Load packages
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import inception_v3, Inception_V3_Weights
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from category_encoders import OrdinalEncoder, OneHotEncoder, TargetEncoder
from sklearn.impute import SimpleImputer
from pathlib import Path
import shared_functions as sf

In [2]:
# Define model & file name
model_name = 'MultiModalModel_2IMG'
file_name = 'property-sales_new-york-city_2022_pre-processed'

In [3]:
# Create output directory for exports
Path(f'../models/{model_name}').mkdir(parents=True, exist_ok=True)

In [4]:
# Load subset keys as list
subset_keys = pd.read_csv(f'../data/processed/subset_keys.csv').squeeze().to_list()

In [5]:
# Load subset index as series
subset_index = pd.read_csv(f'../data/processed/subset_index.csv', index_col=0)

In [6]:
# Use GPU when possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type
print(f'Device type: {device.upper()}')

Device type: CUDA


In [7]:
# Set random seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [8]:
dataset_params = {
    'data': f'../data/processed/{file_name}.parquet',
    'target_name': 'sale_price',
    'to_drop': 'sale_price_adj',
    'image_directory': '../data/raw/satellite-images_new-york-city_2022_640x640_19/',
    'image_transformation': transforms.Compose([
        transforms.CenterCrop((600, 600)), # crop image borders by margin of 20px to remove text from 640x640
        transforms.Resize((299, 299)), # resize image to 299x299
        transforms.ToTensor(),  # convert image to PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # noarmlize based on ImageNet data
        ]),
    'subset_index': '../data/processed/subset_index.csv',
    'input_scaler': StandardScaler(),
    'target_scaler': None,
    'categorical_encoder': TargetEncoder(),
    'numerical_imputer': SimpleImputer(missing_values=pd.NA, strategy='mean'),
    'data_overview': f'../data/processed/{file_name}_data-overview.csv'
    }

In [9]:
# Instantiate datasets
subsets = {subset_key: sf.MultiModalDataset(**dataset_params, subset=subset_key) for subset_key in subset_keys}
dataset = sf.MultiModalDataset(**dataset_params)

In [10]:
# Define model architecture
class MultiModalModel_2IMG(nn.Module):
    # Define model components
    def __init__(self):
        super().__init__()

        # Define text model
        self.TextModel = nn.Sequential(
            nn.Linear(dataset.X_text.shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
            )
        
        # Load optimal weights
        self.TextModel.load_state_dict(torch.load('../models/MLPModel/state_dict.pt'))
        
        # Define image model 1
        self.ImageModel1 = inception_v3(weights=Inception_V3_Weights.DEFAULT)
        self.ImageModel1.aux_logits = False
        for parameter in self.ImageModel1.parameters():
            parameter.requires_grad = False
        self.ImageModel.fc = nn.Sequential(
            nn.Linear(self.ImageModel.fc.in_features, 512),
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
            )
        
        # Define image model 2
        self.ImageModel2 = inception_v3(weights=Inception_V3_Weights.DEFAULT)
        self.ImageModel2.aux_logits = False
        for parameter in self.ImageModel2.parameters():
            parameter.requires_grad = False
        self.ImageModel.fc = nn.Sequential(
            nn.Linear(self.ImageModel.fc.in_features, 512),
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
            )

        # Define linear layer for output
        self.linear = nn.Linear(3, 1)

        # Define acitvation function
        self.relu = nn.ReLU(inplace=True)
    
    # Define forward pass
    def forward(self, X_text, X_image_1, X_image_2):
        X_text = self.relu(self.TextModel(X_text))
        X_image_1 = self.relu(self.ImageModel1(X_image_1))
        X_image_2 = self.relu(self.ImageModel2(X_image_2))
        y = self.linear(torch.cat((X_text, X_image_1, X_image_2), dim=1))
        return y

In [11]:
# Instantiate model
model = MultiModalModel_2IMG().to(device)

In [12]:
# Calculate number of model parameters
n_params = sum(parameter.numel() for parameter in model.parameters())
print(f'# model paramters: {n_params}')

# model paramters: 25118093


In [ ]:
# Do not train if already trained
if Path(f'../models/{model_name}/state_dict.pt').is_file() and Path(f'../models/{model_name}/history.csv').is_file():
    # Load optimal weights and history
    model.load_state_dict(torch.load(f'../models/{model_name}/state_dict.pt'))
    history = pd.read_csv(f'../models/{model_name}/history.csv', index_col=0)
    print('Skipping training and loading optimal weights from previous training!')
else:
    # Train model
    model, history = sf.train_model(
        model=model,
        dataset_train=subsets['train'],
        dataset_val=subsets['val'],

        # Define loss & optimizer
        loss_function=nn.MSELoss().to(device),
        optimizer=optim.Adam(params=model.parameters(), lr=0.01),

        # Define computing device
        device=device,

        # Define training parameters
        epochs=20,
        patience=3,
        delta=0,
        batch_size=64,
        shuffle=True,
        num_workers=0,
        pin_memory=True,

        # Define save locations
        save_state_dict_as=f'../models/{model_name}/state_dict.pt',
        save_history_as=f'../models/{model_name}/history.csv'
        )

In [22]:
# Generate model predictions
predictions = sf.get_predictions(model, dataset, subset_index, device, save_as=f'../models/{model_name}/predictions.csv')

TypeError: to() received an invalid combination of arguments - got (list), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)


In [ ]:
# Compute performance metrics
metrics = sf.get_metrics(predictions, subset_keys, save_as=f'../models/{model_name}/perf_metrics.csv')

In [23]:
# Plot training history
sf.plot_history(history, save_as=f'../models/{model_name}/history.pdf')

AttributeError: 'DataFrame' object has no attribute 'loss_train'

Error in callback <function _draw_all_if_interactive at 0x7f017973b4c0> (for post_execute):


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 453.6x288 with 1 Axes>

In [ ]:
# Plot predictions vs actuals
sf.plot_pred_vs_actual(predictions, save_as=f'../models/{model_name}/predictions_vs_actuals.pdf')